In [ ]:
pip install datasets

In [ ]:
from datasets import get_dataset_config_names
from datasets import load_dataset
dataset_id = "SetFit/amazon_counterfactual"
# Returns ['de', 'en', 'en-ext', 'ja']
configs = get_dataset_config_names(dataset_id)
# Load English subset
dset = load_dataset(dataset_id, name="en")


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dset = load_dataset(dataset_id, name="en")
train_dataset = dset["train"]

for sample in train_dataset[:5]:
    print(sample)


  0%|          | 0/3 [00:00<?, ?it/s]

text
label
label_text


In [ ]:
 import pandas as pd
dataset = dset["train"]

# Convert the dataset to a Pandas DataFrame
df = dataset.to_pandas()

In [ ]:
df.head()

,text,label,label_text
0,In person it looks as though it would have cos...,1,counterfactual
1,Product received as described in the shipping ...,0,not-counterfactual
2,The handkerchiefs were just what I wanted - so...,0,not-counterfactual
3,I adore Nessa and give her credit because the...,0,not-counterfactual
4,He was very pleased when he saw how neat the c...,0,not-counterfactual


In [ ]:
trueValues=[]
for i in df["label"]:
  trueValues.append(i)
trueValues

In [ ]:
trueValues=list(df["label"])
trueValues

In [ ]:
 import pandas as pd
dataset = dset["test"]

# Convert the dataset to a Pandas DataFrame
df1 = dataset.to_pandas()

In [ ]:
df1.head()

,text,label,label_text
0,The one that came with my TV lasted well over ...,0,not-counterfactual
1,Really don't know what I would do without this.,0,not-counterfactual
2,"""This book doesn't belong in the \best ever\""""...",0,not-counterfactual
3,The sweater is soft and washes well without pi...,0,not-counterfactual
4,My 2 year old fits for the jumpsuit size 3t-4t...,0,not-counterfactual


In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install tweet-preprocessor

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import re
import preprocessor as p
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
corpus = list(df['text'])
corpus[:5]

['In person it looks as though it would have cost a lot more.',
 "Product received as described in the shipping time indicated; couldn't be happier - Thank You!!!",
 'The handkerchiefs were just what I wanted - soft and not too thick, not too thin.',
 "I adore Nessa and give her  credit because there is no way I'd be behaving like a normal teen by catching a train, working a job and gasp!...going without cable in my room or an Ipad - and all the while planning a big celebrity wedding for my Dad and his superstar fiancée.",
 'He was very pleased when he saw how neat the closet looked and how much easier it is for him to get his shoes off these racks without everything else falling off as well.']

In [ ]:
corpus_embeddings = embedder.encode(corpus)
corpus_embeddings

array([[-0.06686687,  0.11132997,  0.02574237, ..., -0.16104732,
        -0.06892762,  0.07328395],
       [-0.03896862,  0.11215757, -0.03006651, ...,  0.00388081,
         0.01678169,  0.08797678],
       [-0.07910055,  0.03977076, -0.00335485, ..., -0.01717644,
        -0.00877995,  0.10698651],
       ...,
       [-0.05679178,  0.04162011, -0.06811243, ...,  0.02039492,
         0.07036675, -0.01698068],
       [ 0.00972221,  0.14367522,  0.00260008, ...,  0.0200909 ,
         0.01604301,  0.02421074],
       [-0.00033769, -0.0049198 , -0.02956469, ..., -0.05071732,
         0.03533047,  0.07034355]], dtype=float32)

In [ ]:
num_clusters = 2
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
cluster_assignment

array([0, 0, 0, ..., 0, 0, 1], dtype=int32)

In [ ]:
cluster_df = pd.DataFrame(corpus, columns = ['corpus'])
cluster_df['cluster'] = cluster_assignment
cluster_df.head()

,corpus,cluster
0,In person it looks as though it would have cos...,0
1,Product received as described in the shipping ...,0
2,The handkerchiefs were just what I wanted - so...,0
3,I adore Nessa and give her credit because the...,0
4,He was very pleased when he saw how neat the c...,0


In [ ]:
clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

Cluster  1
['In person it looks as though it would have cost a lot more.', "Product received as described in the shipping time indicated; couldn't be happier - Thank You!!!", 'The handkerchiefs were just what I wanted - soft and not too thick, not too thin.', "I adore Nessa and give her  credit because there is no way I'd be behaving like a normal teen by catching a train, working a job and gasp!...going without cable in my room or an Ipad - and all the while planning a big celebrity wedding for my Dad and his superstar fiancée.", 'He was very pleased when he saw how neat the closet looked and how much easier it is for him to get his shoes off these racks without everything else falling off as well.', "Good for someone who wants to save on cable and doesn't mind not having all the flashy channels.", "This is DEFINITELY a GREAT BUY... You won't go wrong...", 'With my Amazon prime I received this in free one day instead of the free two day for prime.', 'I was going on a trip from CA to u

In [ ]:
from sklearn.metrics import f1_score

# Calculate F-measure
f_measure = f1_score(trueValues, cluster_assignment, average='weighted')

print("F-measure:", f_measure)

F-measure: 0.6933621752461718


In [ ]:
import numpy as np
def get_accuracy(predictions, Y):
    #print(predictions, Y)
    return np.sum(predictions == Y) / len(Y)

In [ ]:
a=get_accuracy(cluster_assignment,trueValues)
a

0.6874066699850672